In [ ]:
!install.packages('ISLR')

In [27]:
library(ISLR)

In [16]:
!install.packages("boot")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error in !install.packages("boot"): invalid argument type


In [17]:
library(boot)

# Resampling Methods

## Cross Validation

### The Validation Set Approach

It involves randomly dividing the available set of observations into two parts, a **training set** and a **validation set** or **hold-out set**. 

**The model is fit on the training set**, and the fitted model is used to **predict the responses for the observations in the validation set**. The resulting validation
set error rate---typically assessed using **MSE** in the case of a quantitative response---provides an estimate of the test error rate.

$$
MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{f}(x_i))^2
$$

In [3]:
set.seed(1)

In [7]:
train=sample(392,196) #split the set by selecting a random subset of 196 observations out of the original 392 

In [9]:
lm.fit = lm(mpg~horsepower, data = Auto, subset = train)

In [10]:
attach(Auto)
mean((mpg-predict(lm.fit,Auto))[-train]^2)

[1] 23.85104

In [12]:
lm.fit2 = lm(mpg~poly(horsepower, 2), data = Auto, subset = train)
mean((mpg-predict(lm.fit2, Auto))[-train]^2)

lm.fit3 = lm(mpg~poly(horsepower, 3), data = Auto, subset = train)
mean((mpg-predict(lm.fit3, Auto))[-train]^2)

[1] 20.86163

[1] 21.12714

### Leave-One-Out Cross-Validation (LOOCV)

LOOCV involves splitting the set of observations into two parts. However, instead of creating two subsets of comparable size, a single observation $(x_1, y_1)$ is used for the validation
set, and the remaining observations $\{(x_2, y_2), \dots , (x_n, y_n)\}$ make up the training set. The statistical learning method is fit on the $n − 1$ training
observations, and a prediction $\hat{y_1}$ is made for the excluded observation, using its value $x_1$. Since $(x_1, y_1)$ was not used in the fitting process, $MSE_1 = (y_1 − \hat{y_1})^2$ provides an approximately unbiased estimate for the test error.

We can repeat the procedure by selecting $(x_2, y_2)$ for the validation data, training the statistical learning procedure on the $n − 1$ observations 
$\{(x_1, y_1), (x_3, y_3), \dots , (x_n, y_n)\}$, and computing $MSE_2 = (y_2−\hat{y_2})^2$. 

Repeating this approach $n$ times produces $n$ squared errors, $MSE_1, \dots , MSE_n$. The **LOOCV** estimate for the test **MSE** is the average of these n test error
estimates:

$$
CV_{(n)} = \frac{1}{n}\sum_{i=1}^{n}MSE_i
$$

In [13]:
glm.fit = glm(mpg~horsepower, data = Auto)
coef(glm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [14]:
lm.fit = lm(mpg~horsepower, data = Auto)
coef(lm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [19]:
glm.fit = glm(mpg~horsepower, data = Auto)
cv.err = cv.glm(Auto, glm.fit)
cv.err$delta

[1] 24.23151 24.23114

In [22]:
cv.error = rep(0,5)
for (i in 1:5) {
    glm.fit = glm(mpg~poly(horsepower, i), data = Auto)
    cv.error[i] = cv.glm(Auto, glm.fit)$delta[1]
}
cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

### k-Fold Cross Validation (k-fold CV)

An alternative to **LOOCV** is k-fold CV. This approach involves randomly dividing the set of observations into $k$ groups, or folds, of approximately equal size. The first fold is treated as a validation set, and the method is fit on the remaining $k − 1$ folds. The mean squared error, $MSE_1$, is then computed on the observations in the held-out fold. This procedure is repeated $k$ times; each time, a different group of observations is treated as a validation set. This process results in $k$ estimates of the test error, $MSE_1,MSE_2, \dots ,MSE_k$. The k-fold CV estimate is computed by averaging
these values,

$$
CV_{(k)} = \frac{1}{k}\sum_{i=1}^{k}MSE_i
$$

In [23]:
set.seed(17)
cv.error.10 = rep(0,10)
for (i in 1:10) {
    glm.fit = glm(mpg~poly(horsepower, i), data = Auto)
    cv.error.10[i] = cv.glm(Auto, glm.fit, K = 10)$delta[1]
}
cv.error.10

[1] 24.27207 19.26909 19.34805 19.29496 19.03198 18.89781 19.12061 19.14666
 [9] 18.87013 20.95520

## The Bootstrap

In [26]:
alpha.fn = function(data, index) {
    X = data$X[index]
    Y = data$Y[index]
    return ((var(Y) - cov(X,Y)) / (var(X) + var(Y) - 2 * cov(X, Y)))
}

In [28]:
alpha.fn(Portfolio, 1:100)

[1] 0.5758321

In [29]:
set.seed(1)
alpha.fn(Portfolio, sample(100, 100, replace = T))

[1] 0.7368375

In [30]:
boot(Portfolio, alpha.fn, R = 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.5758321 -0.001695873  0.09366347

### Estimating the Accuracy of a Linear Regression Model

In [31]:
boot.fn = function(data, index)
    return (coef(lm(mpg~horsepower, data = data, subset = index)))
boot.fn(Auto, 1:392)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [32]:
boot(Auto, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0412152338 0.832986409
t2* -0.1578447 -0.0005105609 0.007228529

In [33]:
summary(lm(mpg~horsepower, data = Auto))$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),39.9358610,0.717498656,55.65984,1.220362e-187
horsepower,-0.1578447,0.006445501,-24.48914,7.031989e-81
